## Loading Libraries

In [1]:
import pandas as pd
import re
import requests
import time
from bs4 import BeautifulSoup as bs

## Generating URLs

In [2]:
from datetime import date, timedelta

# Archive URL
archive_url = 'https://www.thehindubusinessline.com/archive/web/'
dates = []

# Dates for which we will scrape data
#2019/01/08

start_date = date(2018, 10, 29)
end_date = date(2013, 8, 1)
delta = timedelta(days=1)

# This object will store the scraped data
data_dictionary = {}

# Dates will be appended to end of URL
while start_date >= end_date:  
    dates.append(start_date.strftime("%Y/%m/%d"))   
    data_dictionary[start_date.strftime("%Y/%m/%d")] = {}
    start_date -= delta

print(dates)
# print(data_dictionary)

['2018/10/29', '2018/10/28', '2018/10/27', '2018/10/26', '2018/10/25', '2018/10/24', '2018/10/23', '2018/10/22', '2018/10/21', '2018/10/20', '2018/10/19', '2018/10/18', '2018/10/17', '2018/10/16', '2018/10/15', '2018/10/14', '2018/10/13', '2018/10/12', '2018/10/11', '2018/10/10', '2018/10/09', '2018/10/08', '2018/10/07', '2018/10/06', '2018/10/05', '2018/10/04', '2018/10/03', '2018/10/02', '2018/10/01', '2018/09/30', '2018/09/29', '2018/09/28', '2018/09/27', '2018/09/26', '2018/09/25', '2018/09/24', '2018/09/23', '2018/09/22', '2018/09/21', '2018/09/20', '2018/09/19', '2018/09/18', '2018/09/17', '2018/09/16', '2018/09/15', '2018/09/14', '2018/09/13', '2018/09/12', '2018/09/11', '2018/09/10', '2018/09/09', '2018/09/08', '2018/09/07', '2018/09/06', '2018/09/05', '2018/09/04', '2018/09/03', '2018/09/02', '2018/09/01', '2018/08/31', '2018/08/30', '2018/08/29', '2018/08/28', '2018/08/27', '2018/08/26', '2018/08/25', '2018/08/24', '2018/08/23', '2018/08/22', '2018/08/21', '2018/08/20', '2018

## Fetching data for a particular day

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

## The URL
url = 'https://www.thehindubusinessline.com/archive/web/2019/07/31/'

# Request url content
page = requests.get(url, headers = headers)
soup = bs(page.content, 'html.parser')

In [4]:
# Find all Sections for the URL and articles in each section
sections = []
for s in soup.find('div', class_ = 'pad15 margin topStories tp-container').find_all('section'):
    print(s.find('h2').get_text().strip())
    sections.append(s.find('h2').get_text().strip())
    for a in s.find('div', class_= 'col-lg-12 col-md-12 col-sm-12 col-xs-12').find_all('a'):
        #print(a['href'])
        a =1

columns
companies
gold & silver
mobiles & tablets
clean tech
commodities
variety
forex
stocks
social media
markets
agri-biz & commodity
money & banking
today's paper
info-tech
logistics
education
opinion
agri business
technophile
politics
world
news
sports
money & banking
technical analysis
science
national
markets
news
economy
states


## Fetching data for all the urls

In [5]:
## Some helper functions to strip html tags
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

## Example code to get text from article
# Request url content
url = 'https://www.thehindubusinessline.com/companies/hccb-ropes-in-seema-nair-as-edhr/article9711763.ece'
page = requests.get(url, headers = headers)
soup = bs(page.content, 'html.parser')
ps = (soup.find('div', id = 'ControlPara').find_all('p'))

string = ''
for p in ps:
    string += (strip_tags(p.get_text()))
string

'Hindustan Coca-Cola Beverages (HCCB) on Wednesday announced the appointment of Seema Nair as the Executive Director, HR. She succeeds Gaurav Chaturvedi, who has taken over as the Chief Executive Officer of Coca-Cola Pinya Beverages in Myanmar.Chaturvedi is among the 13 associates of Hindustan Coca-Cola, who are serving at senior Executive positions outside India at the Coca-Cola Company.In a statement, Christina Ruggiero, CEO, Hindustan Coca-Cola Beverages, said that the company will be leveraging on Nair’s rich experience to usher in a culture of agility and innovation across its large manufacturing and sales organisation.'

In [6]:
## Function to get article text from links
def get_article_text(s):
    
    print(len(s.find('div', class_= 'col-lg-12 col-md-12 col-sm-12 col-xs-12').find_all('a')))
    articles = []
    
    # For each link in the hindu archive section
    for a in s.find('div', class_= 'col-lg-12 col-md-12 col-sm-12 col-xs-12').find_all('a'):
        
        #print(a['href'])
        # Request url content
        page = requests.get(a['href'], headers = headers)
        soup = bs(page.content, 'html.parser')
        
        # Get the title of article
        title = soup.find('div', class_ = 'title-bor-bottom').find('h1', class_ = 'tp-title-inf')
        
        # Get the paragraphs article
        if soup.find('div', id = 'ControlPara') is not None: 
            paragraphs = (soup.find('div', id = 'ControlPara').find_all('p'))
        
        article_text = ''
        
        # Append text for each paragraph found in html
        for paragraph in paragraphs:
            article_text += (strip_tags(paragraph.get_text()))
        
        if title != None:
            articles.append((title.get_text(), article_text))
            print(title.get_text())
        else:
            articles.append(('Title Not Found', article_text))
            
    return articles

In [ ]:
## Fetching data for all urls

## Define the sections we would like to scrape
sections = set(['companies',
 'columns',
 'gold & silver',
 'mobiles & tablets',
 'clean tech',
 'commodities',
 'forex',
 'stocks',
 'markets',
 'social media',
 'agri-biz & commodity',
 'money & banking',
 "today's paper",
 'info-tech',
 'logistics',
 'education',
 'opinion',
 'agri business',
 'technophile',
 'money & banking',
 'technical analysis',
 'markets',
 'economy']) 

for date in dates:
    
    print(date)
    # Request url content
    page = requests.get(archive_url + date, headers = headers)
    soup = bs(page.content, 'html.parser')

    for section in sections:
        data_dictionary[date][section] = []
        
    # Find all Sections for the URL and articles in each section
    for s in soup.find('div', class_ = 'pad15 margin topStories tp-container').find_all('section'):
        section = (s.find('h2').get_text().strip())
        
        # Get links for each section that I am interested in
        if section in sections:
            print(section)
            data_dictionary[date][section] = get_article_text(s)
        
    # Sleeping for 0.001 seconds (Not required for The Hindu)
    #time.sleep(0.001)

2018/10/29
companies
17
Barmer block: Vedanta gets 10-year extension 
CERC gets 8 weeks to consider tariff changes in Gujarat power plants
Grey Group acquires over 75% stake in Autumn Worldwide
Sterlite Copper accuses TNPCB of providing ‘flawed data’ 
Discom, renewables boost Tata Power’s Q2 profit 
Ex-Hyundai executive Srivastava joins JSW 
GIL to pay Rs 50 lakh to KSPCB for pollution from its plant in Bengaluru 
With recent buys, BigBasket eyes unmanned vending, subscription services 
Century Textiles Q2 net jumps 3-fold to Rs 156 cr 
L&T JV to export missiles to Europe from Coimbatore SEZ
Sundram Fasteners Sept quarter standalone net up 21.8% 
Tata Power Q2 net jumps 85% 
Granules India arm gets 2 observations from USFDA
Prashant Jhawar-led faction agrees to Tata Steel takeover of Usha Martin biz
 Fae Bikes to buy 70 Okinawa Ridge+ e-scooters for Bengaluru operations
#MeToo: Tatas to terminate contract with Suhel Seth’s firm
Sukkhi raises $7 m from Carpediem, Duane Park
columns
1
Wa

In [ ]:
dictionary_df = {}

dictionary_df['Date'] = []
dictionary_df['Section'] = []
dictionary_df['Title'] = []
dictionary_df['Article'] = []

for date, sections in data_dictionary.items():
        
    for section, articles in sections.items():
        
        for article in articles:
            
            dictionary_df['Date'].append(date)
            dictionary_df['Section'].append(section)
            dictionary_df['Title'].append(article[0])
            dictionary_df['Article'].append(article[1])


In [ ]:
df = pd.DataFrame.from_dict(data=dictionary_df)
df.head(10)

In [ ]:
## Remove commas
df.to_csv('data_hbl_2019_07_31_to_2018_10_30.csv', index=False)
df.Title = df.Title.str.replace(',', '')
df.Article = df.Article.str.replace(',', '')
df.to_csv('data_hbl_2019_07_31_to_2018_10_30_comma_removed.csv', index=False)